# Intermediate Examples for Playplot

In [ ]:
import numpy as np
import os
# python -m pip install libfmp
import libfmp.b as lfb
import librosa
import matplotlib.pyplot as plt
from playplot import Session

from example_data import simple_annotations_file, simple_audio_file

sr = 48000
annotations = np.genfromtxt(simple_annotations_file, delimiter=',')
audio, _ = lfb.read_audio(simple_audio_file, mono=True, Fs=sr)

# Example 1
This example shows an alternative way to wrap the plotting function and how the original can be used too.
And the two alternative return types of the plotting function
In addition, the way how to stream audio directly from a file is shown

In [ ]:
def plot(annotations_, disable_cursor=False):
    fig, ax = plt.subplots()
    ax.plot([0, annotations_[-1,0]])
    ax.set_xlabel("number of files played")
    ax.set_ylabel("time in seconds")

    # external plot doesn't need to be interactive
    if disable_cursor:
        return

    # configuration parameters are optional
    return fig, ax


# create session from file
session01 = Session.from_file(simple_audio_file, looping=True)
#place curser at 10s
session01.time = 10

#start the session to enable audio playback
session01.start()

#call the plot function plot will be show normally
plot(annotations)

# wrap the plot function to create a playable version
plot_wrap = session01(plot)## Example 2 Save Animation
# call that playable version
plot_wrap(annotations)
# call playable version with playback disabled
plot_wrap(annotations, disable_cursor=True)

In [ ]:
# It is possible to query the current playback time and state
print(f'Session state: {session01.time=} {session01.paused=}')

# Example 2
This example shows that multiple plotting functions are possible.
How to use extra functions.
The way the "sparse" mapping works (mapping as a list of time position pairs)
and additional plot parameters

An extra cell allows to save the animation frame by frame to be made into a video

In [ ]:
# create session; specify output folder for saved images
session02 = Session(audio, sr, save_folder="animation_out")
session02.start()


@session02
def plot_a(audio_, sr_):
    fig, ax, line = lfb.plot_signal(audio_, sr_, figsize=(12, 4))

    return fig, ax


@session02
def plot_b(audio_, sr_):
    fig, ax = plt.subplots(figsize=(12, 4), dpi=72)

    duration = audio_.shape[0] / sr_
    mapping = mapper(duration)

    ax.plot(mapping[:,1],mapping[:,0])
    ax.update_datalim([(0, duration), (duration, 0)])
    ax.set_xlabel("some position axis")
    ax.set_ylabel("time in seconds")
    plt.tight_layout()

    return fig, ax, {"mapping": mapping,
                     "axvline_kwargs": {"alpha": 0.9, "ls": 'dashdot', "color": 'c', "lw": 2, "zorder": 10},
                     "title": "B02 Sparse Mapping Function",
                     "window_pos": (0, 0)}


def mapper(duration):
    # mapping the midpoint offset
    # after half the audio is played the curser is still at the 10% mark
    mapping = np.array([
        [0, 0],
        [duration / 2, duration / 10],
        [duration, duration]
    ])
    return mapping


# it is possible to use different plotting functions
plot_a(audio, sr)
plot_b(audio, sr)

## Example 2 Save Animation

In [ ]:
# create output folder
if not os.path.exists("animation_out"):
    os.mkdir("animation_out")
# wait until all plots are opened (if plots are already closed wait 10s)
session02.wait_for_plots_opening()

# define parameters
start = 0
end = session02.duration
fps = 30

for i in range(int((end-start)*fps)):
    # jump to the correct time for each frame
    session02.time = i/fps+start
    # save a image (png) for each frame
    session02.save_plot_images(i)

After saving, we can use some ffmpeg magic to create videos, combine them and add audio
```
cd animation_out &&
ffmpeg -framerate 30 -pattern_type glob -i 'Fig*.png' -c:v libx264 -pix_fmt yuv420p -filter_complex "color=white,format=rgb24[c];[c][0]scale2ref[c][i];[c][i]overlay=format=auto:shortest=1,setsar=1" a.mp4 &&
ffmpeg -framerate 30 -pattern_type glob -i 'B02*.png' -c:v libx264 -pix_fmt yuv420p -filter_complex "color=white,format=rgb24[c];[c][0]scale2ref[c][i];[c][i]overlay=format=auto:shortest=1,setsar=1" b.mp4 &&
ffmpeg -i a.mp4 -i b.mp4 -i ../example_data/bach_bwv245_no22_vokalensemble_ilmenau.wav -filter_complex vstack=inputs=2 combined.mp4
```
https://stackoverflow.com/questions/24961127/how-to-create-a-video-from-images-with-ffmpeg
https://stackoverflow.com/questions/11552565/vertically-or-horizontally-stack-mosaic-several-videos-using-ffmpeg/33764934#33764934

# Example 3
This example shows how an extra axes for a cursor could be constructed
It doesn't show any new core features

In [ ]:
session03 = Session(audio, sr)
session03.start()

@session03
def plot():
    from matplotlib.transforms import Bbox

    fig, axs = plt.subplots(2)

    # first subplot
    lfb.plot_signal(audio, sr, ax=axs[0])
    # second subplot
    axs[1].plot(annotations)
    axs[1].set_xlim(annotations[0, 0], annotations[-1, 0])
    axs[1].set_ylabel("some data")

    fig.tight_layout()

    # create bounding box for extra axes
    pa = axs[0].get_position().get_points()
    pb = axs[1].get_position().get_points()
    p = Bbox([[pb[0, 0], pb[0, 1]], [pa[1, 0], pa[1, 1]]])

    # create axes (spanning both plots, with an arbitrary non-zero x-axis)
    ax = fig.add_axes(p, xlim=(0, 1))
    ax.update_datalim([(0, 0), (1, 0)])
    # hide decoration
    ax.axis('off')

    return fig, ax, {"title": "B03 Subplot Cursor"}


plot()

# Example 4:
This example shows how extra elements can be animated and how to register mpl events

In [ ]:
session04 = Session(audio, sr)
session04.start()

@session04
def plot(x):
    fig, ax, line = lfb.plot_signal(x, sr)

    # this element will be animated (is instance of Artist)
    text = ax.text(10, 0, '____', style='italic', bbox={
        'facecolor': 'red', 'alpha': 0.5, 'pad': 10})

    string = ""
    force_redraw = False

    # a simple demo of the mpl key_press_event; write into a text box
    def on_key(event):
        # nonlocal allows to override variables from the outer functions scope similar to global
        nonlocal string, force_redraw
        force_redraw = True
        if event.key == "delete":
            string = ""
        elif event.key == "backspace":
            if len(string) > 0:
                string = string[:-1]
        else:
            string += event.key

    fig.canvas.mpl_connect('key_press_event', on_key)

    last_draw_input = None

    # called every (potential) frame (returns if redraw is needed)
    def draw_function(time, pos, paused):
        nonlocal last_draw_input, force_redraw
        # only redraw if something animated changed
        if last_draw_input == (time, pos, paused) and not force_redraw:
            return False

        last_draw_input = (time, pos, paused)

        text.set_text(f"{time=:3.2f}, {pos=:3.2f}, {paused=} ♯ \N{Music Flat Sign} {string}")

        force_redraw = False
        return True

    # all elements which get animated must be an instance of Artist and returned here
    return fig, ax, {"title": "B04 Extra Animations",
                     "draw_function": draw_function,
                     "artists": [text]}

plot(audio)

# Example 5:
Another example for the draw_function.
This time the plot itself is animated.

In [ ]:
session05 = Session(audio, sr)
session05.start()

@session05
def plot_waveform(x):
    fig, ax, line = lfb.plot_signal(x, sr)
    return fig, ax, {"title": "B05 Waveform"}

plot_waveform(audio)

@session05
def plot_fft(x: np.ndarray):
    N = 8192
    fig, ax = plt.subplots()
    line, = ax.plot(np.arange(N//2+1) * sr / N, np.zeros(N//2+1))
    ax.set_ylim([-80, 50])
    ax.set_xscale('log', base=2)

    ax.set_xlabel("frequency in Hz")
    ax.set_ylabel("power in dB")

    ax.set_xlim(40,10000)

    last_time = 0
    # called every (potential) frame (returns if redraw is needed)
    def draw_function(time, _pos, _paused):
        nonlocal last_time
        # don't redraw if the time has not changed
        if last_time == time:
            return False

        index = int(min(time * sr,x.shape[0] - N))
        frame = x[index:index+N]
        S = librosa.stft(frame,n_fft=N,win_length=N,hop_length=N,center=False)
        S = np.abs(S) ** 2
        S = 10 * np.log10(S + np.finfo(float).eps)

        # update the plot
        line.set_ydata(S)
        last_time = time
        return True

    # update function does nothing, but we need to define it to disable the default behavior
    def update_func(time_, _pos, paused):
        return time_, paused

    # all elements which get animated must be an instance of Artist and returned here
    return fig, ax, {"title": "B05 FFT",
                     "draw_function": draw_function,
                     "artists": [line], # makes the plot animatable
                     "override_update_function": update_func,  # disable default behavior
                     }

plot_fft(audio)